In [1]:
from threading import Lock

lock = Lock()

with lock:
    # 어떤 불변 조건을 유지하면서 작업 수행
    ...
    


Lock 클래서가 with문을 적절히 활성화 해주므로 위 예제는 다음 try/finally 구조와 동등하다


In [2]:
lock.acquire()  

try:
    # 어떤 불변 조건을 유지하면서 작업을 수행한다
    ...
finally:
    lock.release()

이 경우에는 with 문쪽이 더 낫다 try/finally 구조의 반복사용을 피하고 acquire에 대응하는 release 실수로 빠뜨리는 경우를 방지 할 수 있다

contextlib 내장 모듈을 사용하면 객체나 함수를 with 문에서 쉽게 쓸 수 있다

예를 들어 어떤 코드 영역에서 디버깅 관련 로그를 더 많이 남기고 싶다면
다음 코드는 두 단계의 심각성 수준에서 디버깅 로그를 남기는 함수를 정의한다

In [3]:
import logging
def my_function():
    logging.debug('디버깅 데이터')
    logging.error('이 부분은 오류 로그')
    logging.debug('추가 디버깅 데이터 ')
    
my_function() # 프로그램의 디폴드 로그 수준은 Warning이라 이 함수를 실행하면 오류 메시지만 화면에 출력된다

ERROR:root:이 부분은 오류 로그


 컨텍스트 매니저를 정의하면 이 함수의 로그수준을 일시적으로 높일 수 있다

 with 블록을 실행하기전에 로그 심각성 수준을 높이고 블록 실행 직후에 이전 수준으로 회복

In [4]:
from contextlib import contextmanager

@contextmanager
def debug_logging(level):
    logger = logging.getLogger()
    
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield # with 블록의 내용이 실행되는 부분을 저장한다 
    finally:
        logger.setLevel(old_level)
        

In [5]:
with debug_logging(logging.DEBUG):
    print('* 내부: ')
    my_function()
    
print('* 외부:')
my_function()

DEBUG:root:디버깅 데이터
ERROR:root:이 부분은 오류 로그
DEBUG:root:추가 디버깅 데이터 
ERROR:root:이 부분은 오류 로그


* 내부: 
* 외부:


with 문에 전달된 컨텍스트 매니저가 객체를 반환할 수도 있다.

예를 들어 파일을 작성하고 이 파일이 제대로 닫혔는지 확인하려고 한다면
with 문에 open을 전달하면 

open은 with문에서 as를 통해 대상으로 지정된 변수에게 파일 핸들을 전달하고 with 블록에서 나갈때 이 핸들을 닫는다.


In [ ]:
with open('my_output.txt', 'w') as handle:
    handle.write('데이터 입니다!')
    
# 파일 핸들이 열린채로 실행되는 코드의 양을 줄일 수 있다 automatic closed

커스텀 함수가 as 대상 변수에게 값을 제공하도록 하기 위해 필요한 일은 context manager 안에서 yield에 값을 넘기는 것 뿐이다.

In [8]:
@contextmanager
def log_level(level, name):
    logger = logging.getLogger(name)
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield logger
    finally:
        logger.setLevel(old_level)
        
# with에 as 대상 변수로 얻은 로그 객체에 대해 debug와 같은 로그 관련 메서드를 호출하면  
# with 블록 내의 로그 심각성 수준이 낮게 설정돼 있으므로 디버깅 메시지가 출력된다



In [10]:
with log_level(logging.DEBUG, 'my-log') as logger:
    logger.debug(f'대상: {logger.name}!')
    logging.debug('이 메시지는 출력되지 않습니다')
    
logger = logging.getLogger('my-log')
logger.debug('디버그 메시지는 출력되지 않습니다')
logger.error('오류 메시지는 출력됩니다')


DEBUG:my-log:대상: my-log!
ERROR:my-log:오류 메시지는 출력됩니다
